In this notebook we design a multi-modal model using GraphSAGE.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras
from keras.layers import Dense
from keras.models import Sequential
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchtext
import torch
from torch.nn import Linear
import itertools
import nltk
import spacy
import random
from tqdm import tqdm
import time
from IPython.display import Javascript
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, classification_report

In [4]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import math
from collections import Counter
import random
import csv
import os
import sys

In [5]:
econ_df = pd.read_csv('./datasets/nyc_0809.csv')
econ_df.tail(10)

,Unnamed: 0,Own children of the householder 6 to 17 years,YEAR OF ENTRY Foreign born,Median age (years),GROSS RENT Occupied units paying rent Median (dollars),Sex ratio (males per 100 females),"Educational services, and health care and social assistance",relation_max_fee,White,ROOM Median rooms,...,relation_min_fee,HOUSING TENURE Renter-occupied,"Arts, entertainment, and recreation, and accommodation and food services",SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars),SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars),Own children of the householder under 6 years,label,INTPTLONG,INTPTLAT,geoid
1938,1938,776.0,2167.0,40.5,1393.0,95.52,0.27,1756.0,0.76,4.2,...,1105.0,0.33,0.08,698.0,1705.0,531.0,0.0,-74.163658,40.580105,3.608503e+10
1939,1939,530.0,1135.0,41.4,1444.0,93.46,0.29,1556.0,0.79,6.2,...,1105.0,0.23,0.03,910.0,2743.0,265.0,0.0,-74.151510,40.585422,3.608503e+10
1940,1940,608.0,2109.0,42.6,1284.0,87.49,0.30,1556.0,0.73,6.2,...,1024.0,0.22,0.05,845.0,2600.0,425.0,0.0,-74.156982,40.588248,3.608503e+10
1941,1941,488.0,861.0,38.0,1353.0,110.58,0.23,1556.0,0.68,5.5,...,1024.0,0.27,0.12,866.0,2219.0,279.0,0.0,-74.161902,40.590937,3.608503e+10
1942,1942,1062.0,1315.0,40.7,1024.0,111.22,0.19,1793.0,0.88,6.1,...,1284.0,0.24,0.11,930.0,2537.0,424.0,0.0,-74.171136,40.603550,3.608503e+10
1943,1943,1246.0,1239.0,37.0,1555.0,85.26,0.27,1699.0,0.85,6.0,...,427.0,0.19,0.04,715.0,2324.0,583.0,0.0,-74.163503,40.613723,3.608503e+10
1944,1944,786.0,1329.0,36.5,1325.0,94.79,0.23,1699.0,0.59,4.9,...,427.0,0.36,0.10,722.0,2070.0,247.0,0.0,-74.156120,40.618920,3.608503e+10
1945,1945,1184.0,2190.0,36.9,1699.0,100.46,0.28,1606.0,0.54,5.4,...,427.0,0.24,0.03,708.0,2166.0,320.0,0.0,-74.158743,40.623529,3.608503e+10
1946,1946,484.0,500.0,31.1,427.0,65.53,0.21,1699.0,0.25,4.1,...,741.0,0.74,0.06,465.0,2314.0,139.0,1.0,-74.163321,40.630303,3.608503e+10
1947,1947,913.0,868.0,27.8,741.0,75.48,0.46,1699.0,0.30,4.7,...,427.0,0.58,0.16,693.0,2240.0,493.0,1.0,-74.165570,40.637921,3.608503e+10


In [6]:
econ_df.columns

Index(['Unnamed: 0', 'Own children of the householder 6 to 17 years',
       'YEAR OF ENTRY Foreign born', 'Median age (years)',
       'GROSS RENT Occupied units paying rent Median (dollars)',
       'Sex ratio (males per 100 females)',
       'Educational services, and health care and social assistance',
       'relation_max_fee', 'White', 'ROOM Median rooms',
       'Hispanic or Latino (of any race)', 'Mean household income (dollars)',
       'VALUE Median (dollars)', 'Mean travel time to work (minutes)',
       'Total population', 'Rental vacancy rate', 'relation_min_fee',
       'HOUSING TENURE Renter-occupied',
       'Arts, entertainment, and recreation, and accommodation and food services',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars)',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars)',
       'Own children of the householder under 6 years', 'label',
       'INTPTLONG                     

In [7]:
ids = econ_df['geoid']
labels = econ_df['label']
lats = econ_df['INTPTLAT']
longs = econ_df['INTPTLONG                                                                                                                             ']

In [8]:
object_df = pd.read_csv('./datasets/nyc_object_detection.csv')
object_df

,tract_id,house_num,shape_area,shape_length,target
0,36047074200,268.0,1.044916e+06,61292.571045,0
1,36047074600,199.0,1.308087e+06,57184.489429,0
2,36047074800,273.0,1.027715e+06,62545.202877,0
3,36047075000,280.0,1.062852e+06,64652.632935,0
4,36047075200,284.0,6.675883e+05,57595.193994,0
...,...,...,...,...,...
1989,36085015601,2022.0,3.961607e+06,347234.189681,0
1990,36085015602,1148.0,2.407679e+06,204458.362231,0
1991,36085016901,853.0,1.845037e+06,153916.468262,0
1992,36085017005,1221.0,2.951991e+06,230916.151439,0


In [9]:
shortest_df = pd.read_csv('./datasets/nyc_shortest_paths.csv')
shortest_df['id'] = shortest_df['id'].apply(lambda x: int(x[7:]))
shortest_df

,id,hospital_1,hospital_2,subway_1,subway_2,school_1,school_2
0,36005000100,6,7,4,5,2,2
1,36005000200,7,8,5,6,1,1
2,36005000400,8,9,5,6,1,1
3,36005001600,9,10,5,6,1,1
4,36005001900,6,6,1,3,1,1
...,...,...,...,...,...,...,...
2159,36085030302,1,6,17,19,1,1
2160,36085031901,1,6,17,19,1,2
2161,36085031902,2,6,17,19,1,2
2162,36085032300,2,7,18,20,1,1


In [10]:
combined_df = pd.merge(econ_df, object_df, left_on='geoid', right_on='tract_id')
combined_df = pd.merge(combined_df, shortest_df, left_on='geoid', right_on='id')
combined_df

,Unnamed: 0,Own children of the householder 6 to 17 years,YEAR OF ENTRY Foreign born,Median age (years),GROSS RENT Occupied units paying rent Median (dollars),Sex ratio (males per 100 females),"Educational services, and health care and social assistance",relation_max_fee,White,ROOM Median rooms,...,shape_area,shape_length,target,id,hospital_1,hospital_2,subway_1,subway_2,school_1,school_2
0,0,718.0,1681.0,38.6,1237.0,89.77,0.30,1471.0,0.44,5.0,...,3.081239e+06,131998.872615,0,36005000200,7,8,5,6,1,1
1,1,883.0,1346.0,39.4,1390.0,90.81,0.31,1471.0,0.43,5.0,...,1.966456e+06,120481.247835,0,36005000400,8,9,5,6,1,1
2,2,1100.0,2191.0,36.9,945.0,71.75,0.41,1275.0,0.43,4.1,...,2.169841e+06,76275.908720,1,36005001600,9,10,5,6,1,1
3,3,894.0,743.0,27.8,417.0,77.48,0.29,1094.0,0.18,3.6,...,4.306126e+05,13269.447782,1,36005002300,6,6,1,4,1,1
4,4,1151.0,1880.0,30.4,692.0,86.75,0.26,1067.0,0.16,4.0,...,1.147482e+06,40552.412611,0,36005002500,4,4,1,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1943,1943,1246.0,1239.0,37.0,1555.0,85.26,0.27,1699.0,0.85,6.0,...,3.453923e+06,260205.410460,0,36085029104,2,6,17,19,1,1
1944,1944,786.0,1329.0,36.5,1325.0,94.79,0.23,1699.0,0.59,4.9,...,2.409342e+06,162876.772335,0,36085030301,1,5,16,18,1,1
1945,1945,1184.0,2190.0,36.9,1699.0,100.46,0.28,1606.0,0.54,5.4,...,3.024679e+06,202200.303985,0,36085030302,1,6,17,19,1,1
1946,1946,484.0,500.0,31.1,427.0,65.53,0.21,1699.0,0.25,4.1,...,1.173779e+06,62325.617622,1,36085031901,1,6,17,19,1,2


In [11]:
combined_df[['geoid', 'id']]

,geoid,id
0,3.600500e+10,36005000200
1,3.600500e+10,36005000400
2,3.600500e+10,36005001600
3,3.600500e+10,36005002300
4,3.600500e+10,36005002500
...,...,...
1943,3.608503e+10,36085029104
1944,3.608503e+10,36085030301
1945,3.608503e+10,36085030302
1946,3.608503e+10,36085031901


In [12]:
combined_df.columns

Index(['Unnamed: 0', 'Own children of the householder 6 to 17 years',
       'YEAR OF ENTRY Foreign born', 'Median age (years)',
       'GROSS RENT Occupied units paying rent Median (dollars)',
       'Sex ratio (males per 100 females)',
       'Educational services, and health care and social assistance',
       'relation_max_fee', 'White', 'ROOM Median rooms',
       'Hispanic or Latino (of any race)', 'Mean household income (dollars)',
       'VALUE Median (dollars)', 'Mean travel time to work (minutes)',
       'Total population', 'Rental vacancy rate', 'relation_min_fee',
       'HOUSING TENURE Renter-occupied',
       'Arts, entertainment, and recreation, and accommodation and food services',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars)',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars)',
       'Own children of the householder under 6 years', 'label',
       'INTPTLONG                     

In [13]:
ids = combined_df['tract_id']
labels = combined_df['label']
lats = combined_df['INTPTLAT']
longs = combined_df['INTPTLONG                                                                                                                             ']
econ_features = combined_df[['Own children of the householder 6 to 17 years',
       'YEAR OF ENTRY Foreign born', 'Median age (years)',
       'GROSS RENT Occupied units paying rent Median (dollars)',
       'Sex ratio (males per 100 females)',
       'Educational services, and health care and social assistance',
       'relation_max_fee', 'White', 'ROOM Median rooms',
       'Hispanic or Latino (of any race)', 'Mean household income (dollars)',
       'VALUE Median (dollars)', 'Mean travel time to work (minutes)',
       'Total population', 'Rental vacancy rate', 'relation_min_fee',
       'HOUSING TENURE Renter-occupied',
       'Arts, entertainment, and recreation, and accommodation and food services',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars)',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars)',
       'Own children of the householder under 6 years']]
image_features = combined_df[['house_num', 'shape_area']]
path_features = combined_df[['hospital_1', 'hospital_2', 'subway_1', 'subway_2', 'school_1', 'school_2']]

In [14]:
image_features.isnull().any()

house_num      True
shape_area    False
dtype: bool

In [15]:
image_features = image_features.fillna(0)

In [16]:
image_features.isnull().any()

house_num     False
shape_area    False
dtype: bool

In [17]:
nyc_lat_n = 40.913873
nyc_lat_s = 40.499471
nyc_long_w = -74.243356
nyc_long_e = -73.704582	

In [18]:
tracts = pd.DataFrame(list(zip(ids, lats, longs, labels)))
tracts.columns = ['id', 'lat', 'long', 'label']

In [19]:
tracts.describe()

,id,lat,long,label
count,1.948000e+03,1948.000000,1948.000000,1948.000000
mean,3.605328e+10,40.717136,-73.921466,0.251027
std,2.574047e+07,0.085009,0.080784,0.433715
min,3.600500e+10,40.517403,-74.243356,0.000000
25%,3.604702e+10,40.646271,-73.968029,0.000000
50%,3.604710e+10,40.708974,-73.923479,0.000000
75%,3.608102e+10,40.772261,-73.870436,1.000000
max,3.608503e+10,40.913873,-73.704582,1.000000


In [20]:
tracts.tail(10)

,id,lat,long,label
1938,36085027702,40.580105,-74.163658,0.0
1939,36085027704,40.585422,-74.151510,0.0
1940,36085027705,40.588248,-74.156982,0.0
1941,36085027706,40.590937,-74.161902,0.0
1942,36085029103,40.603550,-74.171136,0.0
1943,36085029104,40.613723,-74.163503,0.0
1944,36085030301,40.618920,-74.156120,0.0
1945,36085030302,40.623529,-74.158743,0.0
1946,36085031901,40.630303,-74.163321,1.0
1947,36085031902,40.637921,-74.165570,1.0


In [21]:
for index, row in tracts.iterrows():
    if (row['lat'] > nyc_lat_n or row['lat'] < nyc_lat_s or
            row['long'] > nyc_long_e or row['long'] < nyc_long_w):
        tracts = tracts.drop([index])

In [22]:
def get_distance(p1, p2):
    dist = math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
    return dist

In [23]:
hospital_locations = pd.read_csv('./datasets/nyc_hospitals.csv')['Location 1']
hospital_latlongs = [(latlong.split('\n')[2]).replace('(', '').replace(')', '').replace(' ', '').split(',') for latlong in list(hospital_locations)]

In [24]:
hospitals = []
for h in hospital_latlongs:
    longitude = float(h[1])
    latitude = float(h[0])
    if ((latitude > nyc_lat_s and latitude < nyc_lat_n) and
            (longitude > nyc_long_w and longitude < nyc_long_e)):
        hospitals.append({"longitude": longitude, "latitude": latitude, "label":0, "closest": -1})

In [25]:
for i, hospital in enumerate(hospitals):
    closest_distance = 1000000
    for index, row in tracts.iterrows():
        dist = get_distance((row['long'], row['lat']),
                            (hospital["longitude"], hospital["latitude"]))
        if dist < closest_distance:
            closest_distance = dist
            label = row['label']
            closest_i = index
    hospitals[i]["label"] = int(label)
    hospitals[i]["closest"] = int(closest_i)

In [26]:
subway_locations = pd.read_csv('./datasets/nyc_subway.csv')['the_geom']
subway_latlongs = [s.replace('POINT', '').replace('(', '').replace(')', '').split(' ')[1:3] for s in subway_locations]

In [27]:
subway = []
for s in subway_latlongs:
    longitude = float(s[0])
    latitude = float(s[1])
    if ((latitude > nyc_lat_s and latitude < nyc_lat_n) and
            (longitude > nyc_long_w and longitude < nyc_long_e)):
        subway.append({"longitude": longitude, "latitude": latitude, "label":0, "closest": -1})

In [28]:
for i, station in enumerate(subway):
    closest_distance = 1000000
    for index, row in tracts.iterrows():
        dist = get_distance((row['long'], row['lat']),
                            (station["longitude"], station["latitude"]))
        if dist < closest_distance:
            closest_distance = dist
            label = row['label']
            closest_i = index
    subway[i]["label"] = int(label)
    subway[i]["closest"] = int(closest_i)

In [29]:
school_locations = pd.read_csv('./datasets/nyc_schools.csv')['Location 1']
school_latlongs = [latlong.split('\n')[1].replace('(', '').replace(')', '').replace(' ', '').split(',') for latlong in school_locations[school_locations.notnull()]]

In [30]:
all_schools = []
for s in school_latlongs:
    longitude = float(s[1])
    latitude = float(s[0])
    if ((latitude > nyc_lat_s and latitude < nyc_lat_n) and
            (longitude > nyc_long_w and longitude < nyc_long_e)):
        all_schools.append({"longitude": longitude, "latitude": latitude, "label":0, "closest": -1})

In [31]:
for i, school in enumerate(all_schools):
    closest_distance = 1000000
    for index, row in tracts.iterrows():
        dist = get_distance((row['long'], row['lat']),
                            (school["longitude"], school["latitude"]))
        if dist < closest_distance:
            closest_distance = dist
            label = row['label']
            closest_i = index
    all_schools[i]["label"] = int(label)
    all_schools[i]["closest"] = int(closest_i)

In [32]:
tract_lats = tracts['lat'].tolist()
tract_longs = tracts['long'].tolist()
tract_labels = tracts['label'].tolist()

In [33]:
tract_dicts = []
for i, vals in enumerate(list(zip(tract_longs, tract_lats, tract_labels))):
    t_dict = {"longitude": vals[0], "latitude": vals[1], "label":vals[2], "closest": i}
    tract_dicts.append(t_dict)

In [34]:
all_nodes = tract_dicts + hospitals + subway + all_schools

In [35]:
tract_indices = list(range(0, len(tracts)))
all_indices = list(range(0, len(all_nodes)))
hospital_indices = list(range(len(tracts), len(tracts) + len(hospitals)))
subway_indices = list(range(len(tracts) + len(hospitals), len(tracts) + len(hospitals) + len(subway)))
school_indices = list(range(len(tracts) + len(hospitals) + len(subway), len(tracts) + len(hospitals) + len(subway) + len(all_schools)))

In [36]:
len(hospitals), len(hospital_indices)

(78, 78)

**Creating the graph using NetworkX**

In [37]:
city_graph = nx.Graph()
color_map = []
# Create nodes
for i, node_dict in enumerate(all_nodes):
    city_graph.add_node(i,
                        pos=(node_dict["longitude"],
                             node_dict["latitude"])
                        )
    
    if node_dict["label"] == 1:
         color_map.append('purple')
    else:
        color_map.append('lightblue')

# Add hospital edges
for tract_i in all_indices:
    hospital_dict = {}
    for hospital_i in hospital_indices:
        hospital_dict[hospital_i] = get_distance((all_nodes[hospital_i]["longitude"], all_nodes[hospital_i]["latitude"]),
                                             (all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]))
    
    closest2 = sorted(hospital_dict, key=hospital_dict.get, reverse=False)[0:2]
    
    for i in closest2:
        s_lat = all_nodes[i]["latitude"]
        s_long = all_nodes[i]["longitude"]
        distance = get_distance((all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]),
                                (s_long, s_lat))
        city_graph.add_edge(tract_i, i, weight=distance)

# Add school edges
for tract_i in all_indices:
    school_dict = {}
    for school_i in school_indices:
        school_dict[school_i] = get_distance((all_nodes[school_i]["longitude"], all_nodes[school_i]["latitude"]),
                                             (all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]))
    
    closest2 = sorted(school_dict, key=school_dict.get, reverse=False)[0:2]
    
    for i in closest2:
        s_lat = all_nodes[i]["latitude"]
        s_long = all_nodes[i]["longitude"]
        distance = get_distance((all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]),
                                (s_long, s_lat))
        city_graph.add_edge(tract_i, i, weight=distance)

# Add subway edges
for tract_i in all_indices:
    subway_dict = {}
    for subway_i in subway_indices:
        subway_dict[subway_i] = get_distance((all_nodes[subway_i]["longitude"], all_nodes[subway_i]["latitude"]),
                                             (all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]))
    
    closest2 = sorted(subway_dict, key=subway_dict.get, reverse=False)[0:2]
    
    for i in closest2:
        s_lat = all_nodes[i]["latitude"]
        s_long = all_nodes[i]["longitude"]
        distance = get_distance((all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]),
                                (s_long, s_lat))
        city_graph.add_edge(tract_i, i, weight=distance)

In [38]:
# Create a dictionary of node attributes
attrs = {}
for i, node_dict in enumerate(all_nodes):
    if i in school_indices:
        node_type = [1, 0, 0, 0]
    elif i in hospital_indices:
        note_type = [0, 1, 0, 0]
    elif i in subway_indices:
        node_type = [0, 0, 1, 0]
    else:
        node_type = [0, 0, 0, 1]

    closest_id = node_dict["closest"]

    attrs[i] = {"econ_0": econ_features.iloc[closest_id, 0],
                "econ_1": econ_features.iloc[closest_id, 1],
                "econ_2": econ_features.iloc[closest_id, 2],
                "econ_3": econ_features.iloc[closest_id, 3],
                "econ_4": econ_features.iloc[closest_id, 4],
                "econ_5": econ_features.iloc[closest_id, 5],
                "econ_6": econ_features.iloc[closest_id, 6],
                "econ_7": econ_features.iloc[closest_id, 7],
                "econ_8": econ_features.iloc[closest_id, 8],
                "econ_9": econ_features.iloc[closest_id, 9],
                "econ_10": econ_features.iloc[closest_id, 10],
                "econ_11": econ_features.iloc[closest_id, 11],
                "econ_12": econ_features.iloc[closest_id, 12],
                "econ_13": econ_features.iloc[closest_id, 13],
                "econ_14": econ_features.iloc[closest_id, 14],
                "econ_15": econ_features.iloc[closest_id, 15],
                "econ_16": econ_features.iloc[closest_id, 16],
                "econ_17": econ_features.iloc[closest_id, 17],
                "econ_18": econ_features.iloc[closest_id, 18],
                "econ_19": econ_features.iloc[closest_id, 19],
                "econ_20": econ_features.iloc[closest_id, 20],
                "image_0": image_features["shape_area"][closest_id],
                "image_1": image_features['house_num'][closest_id],
                "path_0": path_features.iloc[closest_id, 0],
                "path_1": path_features.iloc[closest_id, 1],
                "path_2": path_features.iloc[closest_id, 2],
                "path_3": path_features.iloc[closest_id, 3],
                "path_4": path_features.iloc[closest_id, 4],
                "path_5": path_features.iloc[closest_id, 5],              
                "node_type_s": node_type[0],
                "node_type_h": node_type[1],
                "node_type_c": node_type[2],
                "node_type_t": node_type[3]
                }


nx.set_node_attributes(city_graph, attrs)

In [39]:
pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 8.1 MB/s 


In [40]:
import stellargraph as sg
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE

from tensorflow.keras import layers, optimizers, losses, metrics, Model
import tensorflow as tf

from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegressionCV
from sklearn.isotonic import IsotonicRegression

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

from stellargraph.calibration import TemperatureCalibration, IsotonicCalibration
from stellargraph.calibration import plot_reliability_diagram, expected_calibration_error

from stellargraph import datasets
from stellargraph import StellarGraph

In [41]:
attrs_df = pd.DataFrame(attrs).T
attrs_df

,econ_0,econ_1,econ_2,econ_3,econ_4,econ_5,econ_6,econ_7,econ_8,econ_9,...,path_0,path_1,path_2,path_3,path_4,path_5,node_type_s,node_type_h,node_type_c,node_type_t
0,718.0,1681.0,38.6,1237.0,89.77,0.30,1471.0,0.44,5.0,0.75,...,7.0,8.0,5.0,6.0,1.0,1.0,0.0,0.0,0.0,1.0
1,883.0,1346.0,39.4,1390.0,90.81,0.31,1471.0,0.43,5.0,0.65,...,8.0,9.0,5.0,6.0,1.0,1.0,0.0,0.0,0.0,1.0
2,1100.0,2191.0,36.9,945.0,71.75,0.41,1275.0,0.43,4.1,0.67,...,9.0,10.0,5.0,6.0,1.0,1.0,0.0,0.0,0.0,1.0
3,894.0,743.0,27.8,417.0,77.48,0.29,1094.0,0.18,3.6,0.63,...,6.0,6.0,1.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0
4,1151.0,1880.0,30.4,692.0,86.75,0.26,1067.0,0.16,4.0,0.78,...,4.0,4.0,1.0,3.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,155.0,654.0,33.6,1163.0,105.40,0.22,1155.0,0.12,3.7,0.89,...,3.0,4.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0
4310,1919.0,1591.0,26.5,902.0,89.89,0.28,1178.0,0.06,3.6,0.61,...,1.0,6.0,5.0,6.0,1.0,1.0,1.0,0.0,0.0,0.0
4311,1027.0,1266.0,26.0,500.0,74.62,0.34,1063.0,0.09,4.1,0.59,...,5.0,6.0,5.0,6.0,1.0,1.0,1.0,0.0,0.0,0.0
4312,511.0,694.0,35.5,925.0,70.08,0.57,1479.0,0.04,4.1,0.30,...,4.0,6.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0


In [42]:
attrs_df.isnull().any()

econ_0         False
econ_1         False
econ_2         False
econ_3         False
econ_4         False
econ_5         False
econ_6         False
econ_7         False
econ_8         False
econ_9         False
econ_10        False
econ_11        False
econ_12        False
econ_13        False
econ_14        False
econ_15        False
econ_16        False
econ_17        False
econ_18        False
econ_19        False
econ_20        False
image_0        False
image_1        False
path_0         False
path_1         False
path_2         False
path_3         False
path_4         False
path_5         False
node_type_s    False
node_type_h    False
node_type_c    False
node_type_t    False
dtype: bool

In [43]:
scaler = StandardScaler()

In [44]:
# Feed the graph into GraphSAGE
sg_city_graph = StellarGraph.from_networkx(city_graph,
                                           node_features=pd.DataFrame(scaler.fit_transform(attrs_df)))

In [45]:
print(sg_city_graph.info())

StellarGraph: Undirected multigraph
 Nodes: 4314, Edges: 24113

 Node types:
  default: [4314]
    Features: float32 vector, length 33
    Edge types: default-default->default

 Edge types:
    default-default->default: [24113]
        Weights: range=[0, 0.239357], mean=0.0133778, std=0.0196085
        Features: none


In [46]:
labels = [int(node["label"]) for node in all_nodes]
y = np.zeros([len(labels), 2])

for i, val in enumerate(labels):
    if val == 0:
        y[i][0] = 1
        y[i][1] = 0
    elif val == 1:
        y[i][0] = 0
        y[i][1] = 1
    else:
        print("ERROR")

In [47]:
train_set, test_set = model_selection.train_test_split(
    pd.DataFrame(y), train_size=0.8, test_size=None, stratify=pd.DataFrame(y)
)
val_set = test_set

In [48]:
target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_set)
val_targets = target_encoding.fit_transform(val_set)
test_targets = target_encoding.transform(test_set)

In [49]:
batch_size = 150 # Try 200
dim_shape = [16, 4, 2] # Try 32 instead of 16
epochs = 400 # Try 500

In [50]:
for iteration in range(0, 10):
    train_set, test_set = model_selection.train_test_split(
        pd.DataFrame(y), train_size=0.8, test_size=None, stratify=pd.DataFrame(y)
    )
    val_set = test_set
    target_encoding = preprocessing.LabelBinarizer()
    train_targets = target_encoding.fit_transform(train_set)
    val_targets = target_encoding.fit_transform(val_set)
    test_targets = target_encoding.transform(test_set)

    generator = GraphSAGENodeGenerator(sg_city_graph, batch_size, dim_shape)
    train_gen = generator.flow(train_set.index, train_targets)

    graphsage_model = GraphSAGE(
        layer_sizes=dim_shape, generator=generator, bias=True
    )

    x_inp, x_out = graphsage_model.in_out_tensors()
    prediction = layers.Dense(units=train_targets.shape[1], activation="sigmoid")(x_out)

    model = Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.binary_crossentropy,
        metrics=["acc"],
    )

    val_gen = generator.flow(val_set.index, val_targets)
    test_gen = generator.flow(test_set.index, test_targets)

    history = model.fit(
        train_gen, epochs=epochs, validation_data=val_gen, shuffle=True,
    )

    all_gen = generator.flow(list(city_graph.nodes), y)
    all_predictions = model.predict(all_gen).squeeze()[..., np.newaxis]
    all_predictions_df = pd.DataFrame(all_predictions[:,0], index=list(list(city_graph.nodes)))
    test_preds = all_predictions_df.loc[test_set.index, :]

    test_predictions = test_preds.values
    test_predictions_class = ((test_predictions > 0.5) * 1).flatten()
    test_df = pd.DataFrame(
        {
            "Predicted_score": test_predictions.flatten(),
            "Predicted_class": test_predictions_class,
            "True": test_targets[:, 0],
        }
    )
    
    if iteration == 0:
        all_vals_df = pd.DataFrame(
            {
                "Predicted_score": test_predictions.flatten(),
                "Predicted_class": test_predictions_class,
                "True": test_targets[:, 0],
            }
        )
    else:
        all_vals_df = pd.concat([all_vals_df, test_df], ignore_index=True, sort=False)

    print(iteration)

Epoch 1/400
24/24 [==============================] - 33s 986ms/step - loss: 0.6550 - acc: 0.6447 - val_loss: 0.5890 - val_acc: 0.7358
Epoch 2/400
24/24 [==============================] - 13s 541ms/step - loss: 0.5733 - acc: 0.7462 - val_loss: 0.5574 - val_acc: 0.7578
Epoch 3/400
24/24 [==============================] - 12s 482ms/step - loss: 0.5324 - acc: 0.7682 - val_loss: 0.5429 - val_acc: 0.7370
Epoch 4/400
24/24 [==============================] - 12s 489ms/step - loss: 0.5206 - acc: 0.7615 - val_loss: 0.5271 - val_acc: 0.7601
Epoch 5/400
24/24 [==============================] - 12s 490ms/step - loss: 0.5061 - acc: 0.7734 - val_loss: 0.5056 - val_acc: 0.7625
Epoch 6/400
24/24 [==============================] - 12s 484ms/step - loss: 0.4995 - acc: 0.7728 - val_loss: 0.5140 - val_acc: 0.7636
Epoch 7/400
24/24 [==============================] - 12s 494ms/step - loss: 0.4883 - acc: 0.7792 - val_loss: 0.4906 - val_acc: 0.7775
Epoch 8/400
24/24 [==============================] - 12s 488ms

In [51]:
all_vals_df

,Predicted_score,Predicted_class,True
0,0.228407,0,1
1,0.771171,1,1
2,0.936212,1,1
3,0.923158,1,1
4,0.975517,1,1
...,...,...,...
4310,0.946332,1,1
4311,0.912827,1,1
4312,0.061285,0,0
4313,0.060839,0,0


In [52]:
all_vals_df.to_csv("./datasets/nyc_graphsage.csv", encoding='utf-8', index=False)